**MODELLING**
---------

In this notebook, we will build a model to forecast future 3 years of rent based on historical data

In [14]:
from prophet import Prophet
import matplotlib as plt
import pandas as pd
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima 
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

In [15]:
df = pd.read_csv('../data/curated/historical_rental_data.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [16]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing all features
# Drop unnecessary columns
df_cleaned = df.drop(columns=[
    'Year', 'Month', 'Quarter', 
    'Is_Summer', 'Is_Autumn', 'Is_Winter', 'Is_Spring', 'Time_Since_Start',
    'Rent_Lag_1', 'Rent_Lag_3', 'Rent_Lag_12', 'Rent_MA_3', 'Rent_MA_12'
])



# Optionally, use Log_Rent if rent values are highly skewed
# Otherwise, keep using 'Rent'
df_cleaned['Target_Rent'] = df_cleaned['Log_Rent'].fillna(df_cleaned['Rent'])

# Drop rows with NaN in the target rent column
df_cleaned = df_cleaned.dropna(subset=['Target_Rent'])

# Set 'Date' as the time index for SARIMA
#df_cleaned.set_index('Date', inplace=True)

# Function for SARIMA modeling and forecasting for each suburb and property type
def forecast_sarima(df):
    results = []
    no_results = 0

    # Loop over unique Suburb and Property_Type combinations
    unique_combinations = df[['Suburb', 'Property_Type']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        suburb = row['Suburb']
        prop_type = row['Property_Type']
        print(f"Processing: {suburb} - {prop_type}")

        # Subset the data for this suburb and property type
        subset = df[(df['Suburb'] == suburb) & (df['Property_Type'] == prop_type)]

        # Sort data by date and ensure monthly frequency
        subset = subset.sort_values(by='Date').set_index('Date')

        # Handle duplicate dates by aggregating them (e.g., using mean)
        # Only aggregate numeric columns
        numeric_cols = subset.select_dtypes(include=[np.number]).columns
        subset = subset.groupby(subset.index)[numeric_cols].mean()
        
        subset = subset[['Rent']]

        # Handle NaN values (impute or drop)
        subset = subset.fillna(method='ffill').dropna()

        # Ensure data is non-empty and has sufficient length
        if subset.empty or len(subset) < 36:  # Skip if not enough data for a 3-year forecast
            print(f"Skipping {suburb} - {prop_type} due to insufficient data")
            continue

        # Fit SARIMA model
        try:
            model = SARIMAX(subset, order=(1,1,1), seasonal_order=(1,1,1,12))
            sarima_fit = model.fit(disp=False)

            # Forecast the next 36 months (3 years)
            forecast = sarima_fit.get_forecast(steps=36)
            forecast_dates = pd.date_range(subset.index[-1], periods=37, freq='M')[1:]
            forecast_values = forecast.predicted_mean

            # Store results in a DataFrame
            forecast_df = pd.DataFrame({
                'Suburb': suburb,
                'Property_Type': prop_type,
                'Date': forecast_dates,
                'Forecasted_Rent': forecast_values
            })
            results.append(forecast_df)

            # Plot the forecast
            plt.figure(figsize=(10, 6))
            plt.plot(subset.index, subset['Rent'], label='Actual Rent')
            plt.plot(forecast_dates, forecast_values, label='Forecasted Rent', color='orange')
            plt.title(f"SARIMA Rent Forecast for {suburb} - {prop_type}")
            plt.xlabel('Date')
            plt.ylabel('Rent')
            plt.legend()
            
            # Save the plot
            suburb_folder = f"../plots/modelling_plots/{suburb.replace(' ', '_')}/"
            os.makedirs(suburb_folder, exist_ok=True)
            plt.savefig(f"{suburb_folder}{suburb.replace(' ', '_')}_{prop_type.replace(' ', '_')}_forecast.png")
            plt.close()

        except Exception as e:
            print(f"Failed to fit SARIMA model for {suburb} - {prop_type}: {str(e)}")
            no_results += 1

    # Combine all forecast results into a single DataFrame
    if results:
        forecast_results_df = pd.concat(results, axis=0)
        return forecast_results_df
    else:
        raise ValueError(f"No forecasts were generated. {no_results} suburb-property combinations failed.")

# Apply SARIMA forecasting and output forecast results
forecast_df = forecast_sarima(df_cleaned)

# Save forecast results to a CSV file
forecast_df.to_csv('../data/curated/forecast_results.csv', index=False)


Processing: Albert Park-Middle Park-West St Kilda - 1 bedroom flat


/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset = subset.fillna(method='ffill').dropna()
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `

Processing: Armadale - 1 bedroom flat


/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset = subset.fillna(method='ffill').dropna()
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `

Processing: Carlton North - 1 bedroom flat


...

/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:68: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_dates = pd.date_range(subset.index[-1], periods=37, freq='M')[1:]
/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

Processing: Warrnambool - All properties
Processing: Wodonga - All properties


/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset = subset.fillna(method='ffill').dropna()
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `

Processing: Docklands - All properties


/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/maryzhang/myenv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/var/folders/f3/7b141t_576gfjr1gs8rqdfsm0000gn/T/ipykernel_23953/2254524102.py:68: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_dates = pd.date_range(subset.index[-1], periods=37, freq='M')[1:]


In [21]:
# Convert 'Date' column to datetime if it's not already
forecast_df = pd.read_csv("../data/curated/forecast_results.csv")
forecast_df['Date'] = pd.to_datetime(forecast_df['Date'])

# Get the earliest and latest predictions for each combination of suburb and property type
first_rent = forecast_df.groupby(['Suburb', 'Property_Type'])['Forecasted_Rent'].first()
last_rent = forecast_df.groupby(['Suburb', 'Property_Type'])['Forecasted_Rent'].last()

# Calculate the growth rate ((last_rent - first_rent) / first_rent) * 100
growth_rate = ((last_rent - first_rent) / first_rent) * 100

# Group by Suburb and calculate the mean growth rate across all property types
mean_growth_rate = growth_rate.groupby('Suburb').mean()

# Sort by the highest growth rate and get the top 10 unique suburbs
top_10_suburbs = mean_growth_rate.sort_values(ascending=False).head(10)

# Display the result
print("Top 10 Suburbs with Highest Predicted Growth Rate (grouped by suburb):")
print(top_10_suburbs)


Top 10 Suburbs with Highest Predicted Growth Rate (grouped by suburb):
Suburb
Morwell            74.609028
CBD-St Kilda Rd    55.314264
Dromana-Portsea    52.741026
Portland           48.669330
Shepparton         48.626539
Warrnambool        47.058896
Wodonga            45.193578
Sale-Maffra        44.778550
North Bendigo      44.500766
Southbank          42.121326
Name: Forecasted_Rent, dtype: float64


In [18]:
last_rent_df = last_rent.reset_index()

top_10_4_bedroom_houses = last_rent_df[last_rent_df['Property_Type'] == "4 bedroom house"].sort_values(by='Forecasted_Rent', ascending=False).head(10)

print(top_10_4_bedroom_houses)


                                    Suburb    Property_Type  Forecasted_Rent
882                                 Toorak  4 bedroom house      1931.743409
384                                Fitzroy  4 bedroom house      1781.202065
116                               Brighton  4 bedroom house      1779.896952
356                                 Elwood  4 bedroom house      1693.888079
5    Albert Park-Middle Park-West St Kilda  4 bedroom house      1641.719497
809                            South Yarra  4 bedroom house      1608.542029
729                        Prahran-Windsor  4 bedroom house      1483.529968
19                                Armadale  4 bedroom house      1473.146965
342                            Elsternwick  4 bedroom house      1467.651650
534                                Malvern  4 bedroom house      1433.889258


In [19]:
# Reset the index of last_rent to access 'Suburb' and 'Property_Type' as columns
last_rent_df = last_rent.reset_index()

# Filter for '4 bedroom house' and sort the values by 'Forecasted_Rent'
top_10_3_bedroom_houses = last_rent_df[last_rent_df['Property_Type'] == "3 bedroom house"].sort_values(by='Forecasted_Rent', ascending=False).head(10)

# Display the result
print(top_10_3_bedroom_houses)

                                    Suburb    Property_Type  Forecasted_Rent
321                         East Melbourne  3 bedroom house      1305.359485
4    Albert Park-Middle Park-West St Kilda  3 bedroom house      1244.465054
18                                Armadale  3 bedroom house      1202.997734
808                            South Yarra  3 bedroom house      1185.930772
115                               Brighton  3 bedroom house      1183.743753
355                                 Elwood  3 bedroom house      1144.518961
383                                Fitzroy  3 bedroom house      1133.773093
533                                Malvern  3 bedroom house      1114.650873
881                                 Toorak  3 bedroom house      1069.718168
728                        Prahran-Windsor  3 bedroom house      1039.617122


In [20]:
# Reset the index of last_rent to access 'Suburb' and 'Property_Type' as columns
last_rent_df = last_rent.reset_index()

# Filter for '4 bedroom house' and sort the values by 'Forecasted_Rent'
top_10_2_bedroom_houses = last_rent_df[last_rent_df['Property_Type'] == "2 bedroom house"].sort_values(by='Forecasted_Rent', ascending=False).head(10)

# Display the result
print(top_10_2_bedroom_houses)

                         Suburb    Property_Type  Forecasted_Rent
879                      Toorak  2 bedroom house       838.103013
806                 South Yarra  2 bedroom house       834.994609
16                     Armadale  2 bedroom house       818.253497
325               East St Kilda  2 bedroom house       806.739284
388  Fitzroy North-Clifton Hill  2 bedroom house       803.761523
726             Prahran-Windsor  2 bedroom house       800.389720
381                     Fitzroy  2 bedroom house       793.561113
747            Richmond-Burnley  2 bedroom house       785.847610
179               Carlton North  2 bedroom house       782.846442
113                    Brighton  2 bedroom house       779.890421
